# Homework 4: Vector Space Models

## Due Date: Jun 12th
## Total Points: 76 (+ 15 extra credit)
- **Overview**: In this assignment you will implement many of the things you learned in [Chapter 6 of the textbook](https://web.stanford.edu/~jurafsky/slp3/6.pdf). If you haven't read it yet, now would be a good time to do that.  We'll wait.  Done?  Great, let's move on.
    
    We will provide a corpus of Shakespeare plays, which you will use to create a term-document matrix and a term-context matrix. You'll implement a selection of the weighting methods and similarity metrics defined in the textbook. Ultimately, your goal is to use the resulting vectors to measure how similar Shakespeare plays are to each other, and to find words that are used in a similar fashion. All (or almost all) of the code you write will be direct implementations of concepts and equations described in [Chapter 6, sections 6.3-6.7](https://web.stanford.edu/~jurafsky/slp3/6.pdf).

    *All difficulties are easy when they are known.*
- **Delieverables:**
    - Your implementations for the functions in the skeleton code (this notebook)

- **Grading**: We will use the auto-grading system called `PennGrader`. To complete the homework assignment, you should implement anything marked with `#TODO` and run the cell with `#PennGrader` note. **There will be no hidden tests in this assignment.** In other words, you will know your score once you finish all the `#TODO` and run all the `#PennGrader` tests!


## Recommended Readings

- [Vector Semantics](https://web.stanford.edu/~jurafsky/slp3/6.pdf). Dan Jurafsky and James H. Martin. Speech and Language Processing (3rd edition draft)
- [From Frequency to Meaning: Vector Space Models of Semantics](https://www.jair.org/media/2934/live-2934-4846-jair.pdf). Peter D. Turney and Patrick Pantel. Journal of Artificial Intelligence Research 2010
- [Paraphrasing for Style](http://www.aclweb.org/anthology/C12-1177) Wei Xu, Alan Ritter, Bill Dolan, Ralph Grisman, and Colin Cherry. Coling 2012
- [Evaluation methods for unsupervised word embeddings](http://www.aclweb.org/anthology/D15-1036) Tobias Schnabel, Igor Labutov, David Mimno, Thorsten Joachims. EMNLP 2015
- [Community Evaluation and Exchange of Word Vectors at wordvectors.org.](http://www.aclweb.org/anthology/P14-5004) Manaal Faruqui and Chris Dyer. ACL demos 2014

## FAQs

- When finding the top 10 similar items for a given target element, should I count the target element?  
    *No, do not count the target element.*

- How can I represent a character as a vector for calculating similarity?  
    *One reasonable way would be to do it much in the same way as for plays. You would just need to write code to segment out each character as the given code did for each play.*

- What kind of analysis can I perform on the female and male Shakespearean characters for the report?  
    *There can be various ways to go about this. You can look at PCA projections of the vectors representing the characters in the plays to see if the men and women cluster in distinct areas that are demarcated. Or you can look at the similarity scores among women, among men and compare it to the similarity scores between men and men to see if it is significantly different. To account for outliers, you can use averages in this case. Feel free to play around with different vector representations and different similarity measures.*

- How can I improve the performance and efficiency of my code?  
    *Try to use vectorized code wherever possible instead of using loops. You can refer to this resource on [vectorized code](http://www.cs.cornell.edu/courses/cs1112/2015sp/Exams/exam2/vectorizedCode.pdf).*

- How many documents should I consider for reporting similarity scores in the writeup?  
    *You need not report similarity scores for every pair of documents. A subset of similarity scores should be sufficient. For instance, you can include the top 10 of one or two documents.*

- How can I compute the similarity scores on SimLex999 data set, and compute their correlation with human judgments using Kendall's Tau?  
    *You can use simlex data to get a ranking list with your model and calculate the number of concordant and discordant pairs. You can refer to this resource on [Kendall’s Tau](https://www.statisticshowto.datasciencecentral.com/kendalls-tau/).*

## To get started, **make a copy** of this colab notebook into your google drive!

**Understanding Vector Space:**

A vector space is a fundamental concept in mathematics and computer science that provides a way to represent and manipulate objects that have both magnitude and direction. In the context of Natural Language Processing (NLP), we use vector spaces to represent words, documents, or other entities as points in a high-dimensional space.


Vectors:
A vector is an object that has both magnitude (length) and direction.
In NLP, we often represent words or documents as vectors in a high-dimensional space.

Dimensions:
The dimensions of a vector space correspond to the features or characteristics that we use to describe the objects.
For example, in a term-document matrix, each dimension could represent the frequency of a specific word in a document.

Vector Representation:
Each word or document can be represented as a point (vector) in this high-dimensional space.
The position of the point is determined by the values of its features (e.g., word frequencies).

Similarity and Distance:
The similarity between vectors can be measured using various metrics, such as cosine similarity.
Cosine similarity measures the cosine of the angle between two vectors, providing a way to quantify how similar two words or documents are based on their context or content.

Applications:
By representing words and documents as vectors, we can perform various analyses, such as finding similar words, clustering documents, or detecting anomalies.
This representation helps in understanding relationships and patterns in the data.

## Setup 1: PennGrader Setup [4 points]

In [56]:
## DO NOT CHANGE ANYTHING, JUST RUN
%%capture
!pip install penngrader-client

In [57]:
%%writefile notebook-config.yaml

grader_api_url: 'https://23whrwph9h.execute-api.us-east-1.amazonaws.com/default/Grader23'
grader_api_key: 'flfkE736fA6Z8GxMDJe2q8Kfk8UDqjsG3GVqOFOa'

Overwriting notebook-config.yaml


In [58]:
!cat notebook-config.yaml


grader_api_url: 'https://23whrwph9h.execute-api.us-east-1.amazonaws.com/default/Grader23'
grader_api_key: 'flfkE736fA6Z8GxMDJe2q8Kfk8UDqjsG3GVqOFOa'


In [59]:
from penngrader.grader import *

## TODO - Start
STUDENT_ID = 56803282 # YOUR PENN-ID GOES HERE AS AN INTEGER#
## TODO - End

SECRET = STUDENT_ID
grader = PennGrader('notebook-config.yaml', 'CIS5300_OL_23Su_HW4', STUDENT_ID, SECRET)

PennGrader initialized with Student ID: 56803282

Make sure this correct or we will not be able to store your grade


In [60]:
# check if the PennGrader is set up correctly
# do not chance this cell, see if you get 4/4!
name_str = 'Jacky Choi'
grader.grade(test_case_id = 'name_test', answer = name_str)

Correct! You earned 4/4 points. You are a star!

Your submission has been successfully recorded in the gradebook.


## Setup 2: Dataset / Packages
- **Run the following cells without changing anything!**
- [Loading dataset from huggingface](https://huggingface.co/docs/datasets/v1.8.0/loading_datasets.html#from-local-files)

In [61]:
import os
import csv
import subprocess
import re
import random
import numpy as np
from dill.source import getsource

In [62]:
!gdown 1usS5D3heEt6MB60KEXoebuU6QLpwiqhJ # https://drive.google.com/file/d/1usS5D3heEt6MB60KEXoebuU6QLpwiqhJ/view?usp=share_link
!gdown 1RI7YQIA0WUcMUZRY9Frh-Vcet-js2Cd3 # https://drive.google.com/file/d/1RI7YQIA0WUcMUZRY9Frh-Vcet-js2Cd3/view?usp=share_link
!gdown 1c3wLkuSuKkaFIHOQFBK82awMP5H33uBP # https://drive.google.com/file/d/1c3wLkuSuKkaFIHOQFBK82awMP5H33uBP/view?usp=sharing

Downloading...
From: https://drive.google.com/uc?id=1usS5D3heEt6MB60KEXoebuU6QLpwiqhJ
To: /content/play_names.txt
100% 550/550 [00:00<00:00, 1.64MB/s]
Downloading...
From: https://drive.google.com/uc?id=1RI7YQIA0WUcMUZRY9Frh-Vcet-js2Cd3
To: /content/vocab.txt
100% 184k/184k [00:00<00:00, 106MB/s]
Downloading...
From: https://drive.google.com/uc?id=1c3wLkuSuKkaFIHOQFBK82awMP5H33uBP
To: /content/will_play_text.csv
100% 10.3M/10.3M [00:00<00:00, 159MB/s]


In [63]:
######### Helper Functions - DO NOT CHANGE #########
def read_in_shakespeare():
	'''Reads in the Shakespeare dataset processesit into a list of tuples.
		 Also reads in the vocab and play name lists from files.

	Each tuple consists of
	tuple[0]: The name of the play
	tuple[1] A line from the play as a list of tokenized words.

	Returns:
		tuples: A list of tuples in the above format.
		document_names: A list of the plays present in the corpus.
		vocab: A list of all tokens in the vocabulary.
	'''

	tuples = []

	with open('will_play_text.csv') as f:
		csv_reader = csv.reader(f, delimiter=';')
		for row in csv_reader:
			play_name = row[1]
			line = row[5]
			line_tokens = re.sub(r'[^a-zA-Z0-9\s]', ' ', line).split()
			line_tokens = [token.lower() for token in line_tokens]

			tuples.append((play_name, line_tokens))

	with open('vocab.txt') as f:
		vocab =  [line.strip() for line in f]

	with open('play_names.txt') as f:
		document_names =  [line.strip() for line in f]

	return tuples, document_names, vocab

def get_row_vector(matrix, row_id):
	return matrix[row_id, :]

def get_column_vector(matrix, col_id):
	return matrix[:, col_id]
######### Helper Functions - DO NOT CHANGE #########

# Section 1: Term-Document Matrix [14 points]


## 1.1 Creating Term-Document Matrix [10 points]

You will write code to compile a term-document matrix for Shakespeare's plays, following the description in the textbook.

> In a *term-document matrix*, each row represents a word in the vocabulary and each column represents a document from some collection. The figure below shows a small selection from a term-document matrix showing the occurrence of four words in four plays by Shakespeare. Each cell in this matrix represents the number of times a particular word (defined by the row) occurs in a particular document (defined by the column). Thus *clown* appeared 117 times in *Twelfth Night

|             | As You Like It |  Twelfth Night  | Julias Caesar | Henry V |
| :---------: |:--------------:| :-------------: | :----------:  | :-----: |
| **battle**	| 1 | 1 | 8 | 15 |
| **soldier**	| 2 | 2 | 12 | 36 |
| **fool**		| 37 | 58 | 1 | 5 |
| **crown**		| 5 | 117 | 0 | 0 |

The dimensions of your term-document matrix will be the number of documents $D$ (in this case, the number of Shakespeare's plays that we give you in the corpus by the number of unique word types $\|V\|$ in that collection.   The columns represent the documents, and the rows represent the words, and each cell represents the frequency of that word in that document.

- **Problem 1.1: Term Document Matrix** [10 points]

    In your code you will write a function to `create_term_document_matrix`.  This will let you be the hit of your next dinner party by being able to answer trivia questions like *how many words did Shakespeare use?*, which may give us a hint to the answer to *How many words did Shakespeare know?*  The table will also tell you how many words Shakespeare used only once.  Did you know that there's a technical term for that?  In corpus linguistics they are called [*hapax legomena*](https://en.wikipedia.org/wiki/Hapax_legomenon), but I prefer the term *singleton*, because I don't like snooty Greek or Latin terms.


In [64]:
def create_term_document_matrix(line_tuples, document_names, vocab):
	'''Returns a numpy array containing the term document matrix for the input lines.

	Inputs:
		line_tuples: A list of tuples, containing the name of the document and
		a tokenized line from that document.
		document_names: A list of the document names
		vocab: A list of the tokens in the vocabulary

	# NOTE: THIS DOCSTRING WAS UPDATED ON JAN 24, 12:39 PM.

	Let m = len(vocab) and n = len(document_names).

	Returns:
		td_matrix: A mxn numpy array where the number of rows is the number of words
				and each column corresponds to a document. A_ij contains the
				frequency with which word i occurs in document j.
	'''

	vocab_to_id = dict(zip(vocab, range(0, len(vocab))))
	docname_to_id = dict(zip(document_names, range(0, len(document_names))))

	# YOUR CODE HERE

	# The create_term_document_matrix function generates a term-document matrix
	# from a list of tokenized lines and their associated document names.
	# It takes three inputs: line_tuples, a list of tuples containing document names and tokenized lines; document_names,
	# a list of all document names; and vocab, a list of all unique tokens in the vocabulary.
	# You could create a mapping from tokens to their respective indices and from document names to their respective indices.
	# What about the dimensions of the matrix? --> it'd correspond to the number of unique tokens (rows) and the number of documents (columns).
	# You can then populate the matrix by iterating through the tokenized lines and updating the matrix entries to reflect the frequency of each token in each document.
	# The resulting matrix is a NumPy array where each element represents the frequency of a token in a document.
	n = len(document_names)
	m = len(vocab)
	td_matrix = np.zeros((m, n))
	for doc_name, line in line_tuples:
			doc_id = docname_to_id[doc_name]
			for token in line:
					if token not in vocab_to_id:
							continue
					token_id = vocab_to_id[token]
					td_matrix[token_id, doc_id] += 1
	return td_matrix

In [65]:
tuples, document_names, vocab = read_in_shakespeare()
term_doc_matrix = create_term_document_matrix(tuples, document_names, vocab)
term_doc_matrix.shape # should be (22602, 36)

(22602, 36)

In [66]:
# PennGrader - DO NOT CHANGE
grader.grade(test_case_id = 'test_q11_td_matrix', answer = term_doc_matrix[:5000, :35]) # we only check partial data

Correct! You earned 10/10 points. You are a star!

Your submission has been successfully recorded in the gradebook.


## 1.2 Use Term-Document Matrix to Compare Documents [4 points]

The term-document matrix will also let us do cool things like figure out which plays are most similar to each other, by comparing the column vectors.  We could even look for outliers to see if some plays are so dissimilar from the rest of the canon that [maybe they weren't authored by Shakespeare after all](https://en.wikipedia.org/wiki/Shakespeare_authorship_question).  

Let's begin by considering the column representing each play.  Each column is a $\|V\|$-dimensional vector.  Let's use some math to define the similarity of these vectors.   By far the most common similarity metric is the cosine of the angle between the vectors.  The cosine similarity metric is defined in Section 6.3 of the textbook.

> The cosine, like most measures for vector similarity used in NLP, is based on the dot product operator from linear algebra, also called the inner product:

> dot-product($\vec{v}, \vec{w}) = \vec{v} \cdot \vec{w} = \sum_{i=1}^{N}{v_iw_i} = v_1w_1 +v_2w_2 +...+v_Nw_N$

> The dot product acts as a similarity metric because it will tend to be high just when the two vectors have large values in the same dimensions. Alternatively, vectors that have zeros in different dimensions (orthogonal vectors) will have a dot product of 0, representing their strong dissimilarity.

> This raw dot-product, however, has a problem as a similarity metric: it favors long vectors. The vector length is defined as

> $\|\vec{v}\| = \sqrt{\sum_{i=1}^{N}{v_i^2}}$

> The dot product is higher if a vector is longer, with higher values in each dimension. More frequent words have longer vectors, since they tend to co-occur with more words and have higher co-occurrence values with each of them. The raw dot product thus will be higher for frequent words. But this is a problem; we would like a similarity metric that tells us how similar two words are regardless of their frequency.

> The simplest way to modify the dot product to normalize for the vector length is to divide the dot product by the lengths of each of the two vectors. This normalized dot product turns out to be the same as the cosine of the angle between the two vectors, following from the definition of the dot product between two vectors $\vec{v}$ and $\vec{w}$ as:

> $\vec{v} \cdot \vec{w} = \|\vec{v}\|\|\vec{w}\| cos \Theta$

> $\frac{\vec{v} \cdot \vec{w}}{\|\vec{v}\|\|\vec{w}\|} =  cos \Theta$

> The cosine similarity metric between two vectors $\vec{v}$ and $\vec{w}$ thus can be computed

> $cosine(\vec{v}, \vec{w}) = \frac{\vec{v} \cdot \vec{w}}{\|\vec{v}\| \|\vec{w}\|} = \frac{\sum_{i=1}^{N}{v_iw_i}}{\sqrt{\sum_{i=1}^{N}{v_i^2}} \sqrt{\sum_{i=1}^{N}{w_i^2}}} $

The cosine value ranges from 1 for vectors pointing in the same direction, through 0 for vectors that are orthogonal, to -1 for vectors pointing in opposite directions. Since our term-document matrix contains raw frequency counts, it is non-negative, so the cosine for its vectors will range from 0 to 1.  1 means that the vectors are identical, 0 means that they are totally dissimilar.  


- **Problem 1.2: Comparing plays**: Please implement `compute_cosine_similarity`, and for each play in the corpus, score how similar each other play is to it.  Which plays are the closet to each other in vector space (ignoring self similarity)?  Which plays are the most distant from each other? [4 points]


In [67]:
def compute_cosine_similarity(vector1, vector2):
	'''Computes the cosine similarity of the two input vectors.

	Inputs:
		vector1: A nx1 numpy array
		vector2: A nx1 numpy array

	Returns:
		A scalar similarity value.
	'''

	# YOUR CODE HERE

	# In this task, you will implement a function to compute the cosine similarity
	# between the column vectors of the term-document matrix, which represents each play.
	# Cosine similarity measures the angle between two vectors, providing a value between 0 and 1,
	# where 1 indicates identical vectors and 0 indicates orthogonal vectors.
	# By computing these similarities, you can determine which plays are most similar and most dissimilar to each other based on their term frequencies.
	# This analysis can help identify plays with similar language use and potentially uncover outliers in the corpus.
	dot_p = vector1.T @ vector2
	vector1 = np.linalg.norm(vector1)
	vector2 = np.linalg.norm(vector2)
	return dot_p / (vector1 * vector2)





In [68]:
# PennGrader - DO NOT CHANGE
grader.grade(test_case_id = 'test_q12_cos_sim', answer = getsource(compute_cosine_similarity))

Correct! You earned 4/4 points. You are a star!

Your submission has been successfully recorded in the gradebook.


# Section 2: Measuring word similarity [10 points]
Next, we're going to see how we can represent words as vectors in vector space.  This will give us a way of representing some aspects of the *meaning* of words, by measuring the similarity of their vectors.

In our term-document matrix, the rows are word vectors.  Instead of a $\|V\|$-dimensional vector, these row vectors only have $D$ dimensions.  Do you think that's enough to represent the meaning of words? *(Spoiler Alert: no!)*


## 2.1 Term-Context Matrix [10 points]
Instead of using a term-document matrix, a more common way of computing word similarity is by constructing a term-context matrix (also called a word-word matrix), where columns are labeled by words rather than documents.  The dimensionality of this kind of a matrix is $\|V\|$ by $\|V\|$.  Each cell represents how often the word in the row (the target word) co-occurs with the word in the column (the context) in a training corpus.  

- **Problem 2:** For this part of the assignment, you should write the `create_term_context_matrix` function.  This function specifies the size word window around the target word that you will use to gather its contexts.  For instance, if you set that variable to be 4, then you will use 4 words to the left of the target word, and 4 words to its right for the context.  In this case, the cell represents the number of times in Shakespeare's plays the column word occurs in +/-4 word window around the row word. [10 points]

    You can now re-compute the most similar words for your test words using the row vectors in your term-context matrix instead of your term-document matrix.  What is the dimensionality of your word vectors now?  Do the most similar words make more sense than before?

In [69]:
def create_term_context_matrix(line_tuples, vocab, context_window_size=1):
	'''Returns a numpy array containing the term context matrix for the input lines.

	Inputs:
		line_tuples: A list of tuples, containing the name of the document and
		a tokenized line from that document.
		vocab: A list of the tokens in the vocabulary

	# NOTE: THIS DOCSTRING WAS UPDATED ON JAN 24, 12:39 PM.

	Let n = len(vocab).

	Returns:
		tc_matrix: A nxn numpy array where A_ij contains the frequency with which
				word j was found within context_window_size to the left or right of
				word i in any sentence in the tuples.
	'''

	vocab_to_id = dict(zip(vocab, range(0, len(vocab))))

	# YOUR CODE HERE

	# In this part of the assignment, you will write a function called create_term_context_matrix to construct a term-context matrix,
	# also known as a word-word matrix. Unlike the term-document matrix, where rows represent words and columns represent documents,
	# a term-context matrix represents how often each word (target word) co-occurs with other words (context words) within a specified window in a corpus.
	# This function will help us better understand word similarity based on their context. Specifically, you will:

	# 1. Create a matrix where both rows and columns are labeled by words from the vocabulary.
	# 2. Populate the matrix with the frequency counts of words co-occurring within a given context window around each target word.
	# 3. For example, if the context window size is 4, for each word in a line, you will consider the 4 words to its left and 4 words to its right as its context.
	# The dimensionality of your word vectors will now be the size of your vocabulary, providing a more nuanced representation of word meaning.
	n = len(vocab)
	tc_matrix = np.zeros((n, n))

	for doc_name, line in line_tuples:
			for i in range(len(line)):
					target_word = line[i] # Get the target word
					if target_word not in vocab_to_id:
							continue
					target_id = vocab_to_id[target_word]
					start = max(0, i - context_window_size)
					end = min(len(line), i + context_window_size + 1)
          #need to skip current target
					for j in range(start, end):
							if j == i:
									continue

							context_word = line[j]
							if context_word not in vocab_to_id:
									continue
							context_id = vocab_to_id[context_word]
							tc_matrix[target_id, context_id] += 1
	return tc_matrix

In [70]:
term_context_matrix = create_term_context_matrix(tuples, vocab, context_window_size = 2)
term_context_matrix.shape # should be (22602, 22602)

(22602, 22602)

In [71]:
# PennGrader - DO NOT CHANGE
test_cases = [
    [0, 0], # disliken, tribe
    [1317, 5124], # the, sword
    [15363, 10070], # love, you
    [4601, 15242], # good, night
    [8961, 6221], # hope, not
]

test_entries = [term_context_matrix[i, j] for i, j in test_cases]
grader.grade(test_case_id = 'test_q2_term_context_matrix', answer = test_entries)

Correct! You earned 10/10 points. You are a star!

Your submission has been successfully recorded in the gradebook.


# Section 3: Weighting terms [28 points]
Your term-context matrix contains the raw frequency of the co-occurrence of two words in each cell.  Raw frequency turns out not to be the best way of measuring the association between words.  There are several methods for weighting words so that we get better results.  You should implement two weighting schemes:

* Positive pointwise mutual information (PPMI)
* Term frequency inverse document frequency (tf-idf)

These are defined in Section 6.2 of the textbook.

*Warning, calculating PPMI for your whole $\|V\|$-by-$\|V\|$ matrix might be slow. Our intrepid TA's implementation for PPMI takes about 10 minutes to compute all values. She always writes perfectly optimized code on her first try. You may improve performance by using matrix operations a la MATLAB.*

In [72]:
# In this part of the assignment, you will implement two weighting schemes—Positive Pointwise Mutual Information (PPMI) and
# Term Frequency-Inverse Document Frequency (tf-idf)—to improve the way we measure word associations in a term-context matrix and a term-document matrix, respectively.

# Why Weighting Schemes?

# Raw Frequency Limitations:
# Using raw frequency to measure word associations often leads to suboptimal results. Frequent words tend to have high counts, which can overshadow
# less frequent but potentially more meaningful associations.

# Positive Pointwise Mutual Information (PPMI):
# PPMI is a method that highlights how much more often words co-occur than we would expect by chance. It reduces the influence of
# very frequent words by comparing the actual co-occurrence with the expected co-occurrence, thus emphasizing meaningful associations.
# In this task, you will compute the PPMI matrix from a given term-context matrix. This involves calculating the pointwise mutual information (PMI)
# and then converting it to PPMI by setting negative PMI values to zero.

# Term Frequency-Inverse Document Frequency (tf-idf):
# tf-idf is a weighting scheme that balances term frequency (how often a word appears in a document) with inverse document frequency
# (how common or rare a word is across all documents). This approach helps to highlight words that are important in a document but not too common in the overall corpus.
# You will transform the term-document matrix using tf-idf weighting, making it more effective for tasks like document similarity or classification.

In [73]:
# Expectations

# PPMI Matrix:
# Implement the create_PPMI_matrix function to convert a term-context matrix into a PPMI matrix.
# Use numpy operations to efficiently calculate the probabilities and PMI values, then set negative PMI values to zero to get PPMI.

# tf-idf Matrix:
# Implement the create_tf_idf_matrix function to transform a term-document matrix using tf-idf weighting.
# Calculate the inverse document frequency (idf) for each term and use it to scale the term frequencies.
# By applying these weighting schemes, you will improve the quality of the word vectors, making them more suitable for measuring word associations and document similarities.

## 3.1 Weighting Schemes [20 points]

- **Problem 3.1.1:** Implementing `create_PPMI_matrix` function [10 points]

- **Problem 3.1.2:** Implementing `create_tf_idf_matrix` function [10 points]

In [74]:
def create_PPMI_matrix(term_context_matrix):
	'''Given a term context matrix, output a PPMI matrix.

	See section 6.2 in the textbook.

	Hint: Use numpy matrix and vector operations to speed up implementation.
				Please also add a small constant 1e-6 to your term_context_matrix to avoid having 0s

	Input:
		term_context_matrix: A nxn numpy array, where n is
				the numer of tokens in the vocab.

	Returns: A nxn numpy matrix, where A_ij is equal to the
		 point-wise mutual information between the ith word
		 and the jth word in the term_context_matrix.
	'''
	sc = 1e-6

	# YOUR CODE HERE
	tc_matrix = term_context_matrix + sc
	total_count = np.sum(tc_matrix)
	P_ij = tc_matrix/total_count
	P_i = np.sum(tc_matrix, axis = 1)/total_count
	P_j = np.sum(tc_matrix, axis = 0)/total_count
	PMI = np.log2(P_ij / np.outer(P_i, P_j))
	PPMI_matrix = np.maximum(PMI, 0)
	return PPMI_matrix


def create_tf_idf_matrix(term_document_matrix):
    """Given the term document matrix, output a tf-idf weighted version.

	See section 6.2 in the textbook, but note the log10 info below

	Hint: Use numpy matrix and vector operations to speed up implementation.

	We use Log10 for TF-IDF, but only apply the log10 to idf, not tf.
	tf should be  `tf = term_document_matrix`

	Input:
		term_document_matrix: Numpy array where each column represents a document
		and each row, the frequency of a word in that document.

	Returns:
		A numpy array with the same dimension as term_document_matrix, where
		A_ij is weighted by the inverse document frequency of document h.
	"""
    tf = term_document_matrix

    df = np.sum(term_document_matrix > 0, axis=1)

    N = term_document_matrix.shape[1]

    idf = np.log10((N) / (df))
    idf = idf.reshape(-1, 1)
    tf_idf = tf * idf

    return tf_idf

- Testing `create_tf_idf_matrix`

In [75]:
term_doc_matrix = create_term_document_matrix(tuples, document_names, vocab)
tfidf_matrix = create_tf_idf_matrix(term_doc_matrix)
tfidf_matrix.shape # should be (22602, 36)

(22602, 36)

In [76]:
# PennGrader - DO NOT CHANGE
grader.grade(test_case_id = 'test_q31_tfidf_matrix', answer = tfidf_matrix[:1001, :31]) # we only check part of the answer so that we won't blow up the RAM

Correct! You earned 10/10 points. You are a star!

Your submission has been successfully recorded in the gradebook.


- Testing `create_PPMI_matrix`

In [77]:
mini_vocab = ['dagger', 'run', 'the', 'bloody', 'sword'] # we only use part of the vocab so that you won't blow up the RAM
term_context_matrix = create_term_context_matrix(tuples, mini_vocab, context_window_size = 2)
PPMI_matrix = create_PPMI_matrix(term_context_matrix)
PPMI_matrix.shape # should be (5, 5)

(5, 5)

In [78]:
# PennGrader - DO NOT CHANGE
grader.grade(test_case_id = 'test_q32_ppmi_matrix', answer = PPMI_matrix)

Correct! You earned 10/10 points. You are a star!

Your submission has been successfully recorded in the gradebook.


## 3.2 Other Similarity Measurements [8 points]
There are several ways of computing the similarity between two vectors.  In addition to writing a function to compute cosine similarity in Section 1, you should also write functions to `compute_jaccard_similarity` and `compute_dice_similarity`. You can check out the defintion of the [Jaccard measure here](https://en.wikipedia.org/wiki/Jaccard_index#Weighted_Jaccard_similarity_and_distance). And, dice similarity measure is given by (2 * J)/(J + 1) where J is Jaccard index. Please refer to this wikipedia link on [Dice coefficient](https://en.wikipedia.org/wiki/S%C3%B8rensen%E2%80%93Dice_coefficient) for more details.


- **Problem 3.2.1:** Implementing `jaccard` similarity [4 points]

In [79]:
def compute_jaccard_similarity(vector1, vector2):
	'''Computes the cosine similarity of the two input vectors.

	Inputs:
		vector1: A nx1 numpy array
		vector2: A nx1 numpy array

	Returns:
		A scalar similarity value.
	'''

	# YOUR CODE HERE
	intersection = np.sum(np.minimum(vector1, vector2))
	union = np.sum(np.maximum(vector1, vector2))
	return intersection / union

In [80]:
# PennGrader - DO NOT CHANGE
grader.grade(test_case_id = 'test_q33_jaccard_sim', answer = getsource(compute_jaccard_similarity))

Correct! You earned 4/4 points. You are a star!

Your submission has been successfully recorded in the gradebook.


- **Problem 3.2.2:** Implementing `dice` similarity [4 points]

In [81]:
def compute_dice_similarity(vector1, vector2):
	'''Computes the cosine similarity of the two input vectors.

	Inputs:
		vector1: A nx1 numpy array
		vector2: A nx1 numpy array

	Returns:
		A scalar similarity value.
	'''

	# YOUR CODE HERE
	#couldnt call helper function..
	intersection = np.sum(np.minimum(vector1, vector2))
	union = np.sum(np.maximum(vector1, vector2))

	jaccard = intersection / union

 	#(2 * J)/(J + 1)
	dice = (2 * jaccard)/(jaccard + 1)

	return dice


In [82]:
# PennGrader - DO NOT CHANGE
grader.grade(test_case_id = 'test_q34_dice_sim', answer = getsource(compute_dice_similarity))

Correct! You earned 4/4 points. You are a star!

Your submission has been successfully recorded in the gradebook.


# Section 4: Ranking [10 points]
In this section, you will put everything together and **rank the playes and the words** based on their similarity to others.
- **Problem 4.1:** Implement `rank_plays` [5 points]

In [83]:
# In this section, you are required to implement two functions: rank_plays and rank_words.

# The rank_plays function should take a list of document names, an index representing the target play,
# a term-document matrix, and a similarity function. It will rank all plays in the corpus by their similarity
# to the target play using the specified similarity metric and return a list of play names ordered by decreasing similarity.

# Similarly, the rank_words function should take a list of vocabulary terms, an index representing the target word, a term-context matrix,
# and a similarity function. It will rank all words in the vocabulary by their similarity to the target word using the specified similarity metric
# and return a list of words ordered by decreasing similarity.

# These functions will help identify the most similar plays and words based on their vector representations.

In [84]:
def rank_plays(target_play_index, term_document_matrix, similarity_fn):
	''' Ranks the similarity of all of the plays to the target play.

	Inputs:
		target_play_index: The integer index of the play we want to compare all others against.
		term_document_matrix: The term-document matrix as a mxn numpy array.
		similarity_fn: Function that should be used to compared vectors for two
			documents. Either compute_dice_similarity, compute_jaccard_similarity, or
			compute_cosine_similarity.

	Returns:
		A length-n list of indices corresponding to play names,
		ordered by decreasing similarity to the play indexed by target_play_index,
		storing in a numpy array.
	'''
	m, n = term_document_matrix.shape

	# YOUR CODE HERE
	target_play_vector = term_document_matrix[:, target_play_index]
	sims_sort = np.argsort([similarity_fn(target_play_vector, term_document_matrix[:, i]) for i in range(n)])[::-1]

	return sims_sort

In [85]:
print(rank_plays(0, term_doc_matrix, compute_cosine_similarity))
#rank play by name
for i in rank_plays(0, term_doc_matrix, compute_cosine_similarity):
    print(document_names[i])


[ 0 15 16 11 29 24 27  6 33 22 32 26 17 10  7  2 20 14 12  1 13  9 34  4
 31 28 30 35 18 25 21  5 19  3  8 23]
Henry IV
The Tempest
Henry VI Part 2
King John
Merchant of Venice
Henry VI Part 1
Troilus and Cressida
Hamlet
King Lear
Richard III
Cymbeline
Henry V
As you like it
Richard II
A Midsummer nights dream
Loves Labours Lost
Henry VIII
A Winters Tale
macbeth
Alls well that ends well
Timon of Athens
Romeo and Juliet
Pericles
Antony and Cleopatra
Othello
Twelfth Night
Henry VI Part 3
Titus Andronicus
Julius Caesar
Much Ado about nothing
Measure for measure
Coriolanus
A Comedy of Errors
Taming of the Shrew
Merry Wives of Windsor
Two Gentlemen of Verona


In [86]:
# DO NOT CHANGE #
term_doc_matrix = create_term_document_matrix(tuples, document_names, vocab)
tfidf_matrix = create_tf_idf_matrix(term_doc_matrix)

In [87]:
# Test Case - DO NOT CHANGE #
target_play = 'Richard III'
play_index = document_names.index(target_play)
target_play_ranking = rank_plays(play_index, tfidf_matrix, compute_cosine_similarity)
target_play_ranking.shape # should be a list of 36 numbers, representing the sorted similarity compared to the target play, Richard III

(36,)

In [88]:
# PennGrader - DO NOT CHANGE
grader.grade(test_case_id = 'test_q41_play_ranking', answer = target_play_ranking)

Correct! You earned 5/5 points. You are a star!

Your submission has been successfully recorded in the gradebook.


- **Problem 4.2:** Implement `rank_plays` [5 points]

In [89]:
def rank_words(target_word_index, matrix, similarity_fn):
	''' Ranks the similarity of all of the words to the target word.

	Inputs:
		vocab: List of terms, corresponding to target_word_index rows (i.e. word corresponding
			to target_word_index[i,:] is given by vocab[i])
		target_word_index: The index of the word we want to compare all others against.
		matrix: Numpy matrix where the ith row represents a vector embedding of the ith word.
		similarity_fn: Function that should be used to compared vectors for two word
			ebeddings. Either compute_dice_similarity, compute_jaccard_similarity, or
			compute_cosine_similarity.

	Returns:
		A length-n list of indices, ordered by decreasing similarity to the
		target word indexed by word_index, storing in a numpy array.
	'''

	# YOUR CODE HERE
	target_word_vector = matrix[:, target_word_index]
	sims_sort = np.argsort([similarity_fn(target_word_vector, matrix[:, i]) for i in range(matrix.shape[1])])[::-1]
	return sims_sort

In [90]:
# Test Case - DO NOT CHANGE #
N = 6000
tmp_vocab = vocab[:N] # we only use part of the vocab so that you won't blow up the RAM
term_context_matrix = create_term_context_matrix(tuples, tmp_vocab, context_window_size = 2)
tmp_PPMI_matrix = create_PPMI_matrix(term_context_matrix)

target_word = 'sword'
i = tmp_vocab.index(target_word)

target_word_ranking = rank_words(i, tmp_PPMI_matrix, compute_cosine_similarity)

In [91]:
# PennGrader - DO NOT CHANGE
grader.grade(test_case_id = 'test_q42_word_ranking', answer = target_word_ranking[:5])

Correct! You earned 5/5 points. You are a star!

Your submission has been successfully recorded in the gradebook.


# Section 5: Free-response Questions [10 points]

In the ranking tasks, play with different vector representations and different similarity functions. **Does one combination appear to work better than another? Do any interesting patterns emerge?**

Some patterns you could touch upon (you can pick 1 aspect to answer):
* The fourth column of `will_play_text.csv` contains the name of the character who spoke each line. Using the methods described above, **which characters are most similar? Least similar? **
* Shakespeare's plays are traditionally classified into [comedies, histories, and tragedies](https://en.wikipedia.org/wiki/Shakespeare%27s_plays). **Can you use these vector representations to cluster the plays?**
* Do the vector representations of **[female characters](https://en.wikipedia.org/wiki/Category:Female_Shakespearean_characters)** differ distinguishably from **[male ones](https://en.wikipedia.org/wiki/Category:Male_Shakespearean_characters)**?

I compared the similarity of characters from the 4th column of the csv file and their vocab using cosine similarity, dice and jaccard. I tested one of the characters I knew more about like "Romeo". I found out that these similarities are not very accurate. I think the problems with this comparison is that it is comparing the individual words spoken by the characters and not how they are speaking the words which creates a false illusion that the characters are speaking the same way. I don't think Romeo is similar to Ligarius at all and I am suprised Juliet did not make it in top 5 similarities because they share similar personality traits.

I tried out other similarty functions like Dice and saw that it gave a more accurate representation showing Juliet is one of the most similar to Romeo. I believe its because Dice is more directly focused on the vocab and maybe Romeo and Juliet use similar words when speaking.

In [92]:
def get_character_names(csv_file):
    character_names = set()
    with open(csv_file, 'r') as file:
        reader = csv.reader(file, delimiter=';')
        next(reader)
        for row in reader:
            character_name = row[4].strip()
            character_names.add(character_name.lower())
    return list(character_names)

In [93]:
def create_cd_matrix(line_tuples, document_names, character_names):
    num_characters = len(character_names)
    num_documents = len(document_names)
    cd = np.zeros((num_characters, num_documents))
    character_to_index = {char: i for i, char in enumerate(character_names)}
    document_to_index = {doc: i for i, doc in enumerate(document_names)}
    for play_name, line_tokens in line_tuples:
        if line_tokens:
            character_name = line_tokens[0]
            if character_name in character_to_index and play_name in document_to_index:
                char_index = character_to_index[character_name]
                doc_index = document_to_index[play_name]
                cd[char_index, doc_index] += 1
    return cd

In [94]:
def create_cc_matrix(line_tuples, character_names, vocab, context_window_size=2):
    num_characters = len(character_names)
    num_vocab = len(vocab)
    cc_matrix = np.zeros((num_characters, num_vocab))

    character_to_index = {char: i for i, char in enumerate(character_names)}
    vocab_to_index = {word: i for i, word in enumerate(vocab)}

    for play_name, line_tokens in line_tuples:
        if line_tokens:
            character_name = line_tokens[0]

            if character_name in character_to_index:
                char_index = character_to_index[character_name]
                for i in range(len(line_tokens)):
                    if line_tokens[i] in vocab_to_index:
                        word_index = vocab_to_index[line_tokens[i]]
                        cc_matrix[char_index, word_index] += 1

    return cc_matrix

In [95]:
def similarity_chars(target_character_index, matrix, similarity_fn):
    num_characters = matrix.shape[0]
    target_character_vector = matrix[target_character_index, :]
    similarity_scores = []
    for i in range(num_characters):
        if i != target_character_index:
            similarity_scores.append(similarity_fn(target_character_vector, matrix[i, :]))
        else:
            similarity_scores.append(0)
    sorted_indices = np.argsort(similarity_scores)[::-1]
    return sorted_indices

In [96]:
character_names = get_character_names('will_play_text.csv')
print(character_names)

['', 'second brother', 'silvia', 'first goth', 'su ffolk', 'gardiner', 'brandon', 'lady', 'surveyor', 'siward', 'romeo', 'tranio', 'john of gaunt', 'soldier', 'clifford', 'prince', 'erpingham', 'lodovico', 'herbert', 'iago', 'second guard', 'mother', 'nurse', 'egeus', 'launce', 'first fisherman', 'timon', 'katharine', 'watchman', 'provost', 'cerimon', 'fourth gentleman', 'first brother', 'gremio', 'calchas', 'longaville', 'first neighbour', 'all lords', 'third stranger', 'menecrates', 'mercutio', 'mayor', 'buckingham', 'othello', 'prince edward', 'first attendant', 'sir hugh evans', 'stephano', 'puck', 'touchstone', 'king lear', 'scroop', 'beatrice', 'dion', 'cupid', 'both citizens', 'luciana', 'ophelia', 'silius', 'second clown', 'cade', 'lord willoughby', 'theseus', 'french soldier', 'lord berkeley', 'oswald', 'alonso', 'queen elinor', 'calpurnia', 'lady macduff', 'mistress overdone', 'rivers', 'painter', 'lady faulconbridge', 'blunt', 'bishop of ely', 'spirit', 'third bandit', 'firs

In [97]:
cd_matrix = create_cd_matrix(tuples, document_names, character_names)
cc_matrix = create_cc_matrix(tuples, character_names, vocab, context_window_size=2)

In [99]:
#cosine
name = 'romeo'
name = name.lower()
target_character_index = character_names.index(name)
similar_chars_indices = similarity_chars(target_character_index, cd_matrix, compute_cosine_similarity)
print("Most similar\n")
for index in similar_chars_indices[:5]:
    print(character_names[index])

print("\nLeast similar\n")
for index in similar_chars_indices[-5:]:
    print(character_names[index])

Most similar

lady percy
first executioner
first ambassador
lennox
le beau

Least similar

goneril
trebonius
charles
master
rosalind


<ipython-input-67-df4f3884d5d8>:23: RuntimeWarning: invalid value encountered in scalar divide
  return dot_p / (vector1 * vector2)


In [104]:
#dice
similar_chars_indices = similarity_chars(target_character_index, cd_matrix, compute_dice_similarity)
print("Most similar\n")
for index in similar_chars_indices[:5]:
    print(character_names[index])

print("\nLeast similar\n")
for index in similar_chars_indices[-6:]:
    print(character_names[index])

Most similar

tybalt
nurse
juliet
peter
mercutio

Least similar

some speak
cressida
lorenzo
queen margaret
second fisherman



In [105]:
#jaccard
similar_chars_indices = similarity_chars(target_character_index, cd_matrix, compute_jaccard_similarity)
print("Most similar\n")
for index in similar_chars_indices[:5]:
    print(character_names[index])

print("\nLeast similar\n")
for index in similar_chars_indices[-6:]:
    print(character_names[index])

Most similar

tybalt
nurse
juliet
peter
mercutio

Least similar

some speak
cressida
lorenzo
queen margaret
second fisherman



# Extra credit [15 points]

Quantifying the goodness of one vector space representation over another can be very difficult to do.  It might ultimately require testing how the different vector representations change the performance when used in a downstream task like question answering. A common way of quantifying the goodness of word vectors is to use them to compare the similarity of words with human similarity judgments, and then calculate the correlation of the two rankings.

If you would like extra credit on this assignment, you can quantify the goodness of each of the different vector space models that you produced (for instance by varying the size of the context window, picking PPMI or tf-idf, and selecting among cosine, Jaccard, and Dice).  You can calculate their scores on the [SimLex999 data set](https://www.cl.cam.ac.uk/~fh295/simlex.html), and compute their correlation with human judgments using [Kendall's Tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient).

- **Deliverables**:
    - In the following cells, explain what **experiments** you ran, and which **settings** *(i.e. size of the context window, picking PPMI or tf-idf, and selecting among cosine, Jaccard, and Dice)* had the highest correlation with human judgments.

- **Hint**:
    - For SimLex999 dataset, you only need to use `word1`, `word2`, and `SimLex999` columns, where `SimLex999` column is a **similarity rating by human beings** (ranging from 0 to 10) between `word1` and `word2`.
    - For Kendal's Tau, you can use check [this python implementation](https://www.geeksforgeeks.org/python-kendall-rank-correlation-coefficient/#) for reference.



In [106]:
# Approach:

# 1. Prepare the Data:

# Use the SimLex999 dataset, which provides human similarity ratings for word pairs.
# Focus on the columns: word1, word2, and SimLex999 (similarity ratings by humans).

# 2. Generate Vector Representations:

# Create different term-context matrices by varying the size of the context window.
# Apply different weighting schemes like PPMI and tf-idf to these matrices.
# Compute word vectors using these weighted matrices.

# 3. Compute Similarities:

# For each pair of words in the SimLex999 dataset, calculate their similarity using different metrics (cosine, Jaccard, and Dice) based on the generated word vectors.

# 4. Calculate Correlation:

# Compare the computed similarities with human judgments from the SimLex999 dataset.
# Calculate the correlation between the model's similarity scores and human judgments using Kendall's Tau.

# 5. Report Findings:

# Run experiments with different settings and record which configurations (context window size, weighting scheme, and similarity metric) yield
# the highest correlation with human judgments.

# Submission
### Congratulation on finishing your homework! Here are the deliverables you need to submit to GradeScope
- This notebook and py file: rename to `homework4.ipynb` and `homework4.py`. You can download the notebook and py file by going to the top-left corner of this webpage, `File -> Download -> Download .ipynb/.py`